In [22]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
import joblib
from catboost.utils import get_confusion_matrix



In [23]:
scores=pd.read_csv('scores_enhanced.csv')
scores = scores.drop(["goles_local", "goles_visitante", "fecha", "gana_local", 'empate', 'gana_visitante', 'arbitro'], axis=1)

scores.sample(5)

,year,torneo,fase,jornada,local,visitante,dia,arbitro,pos_local,pos_visitante,...,los_visit_visit_3_matches,vict_local_igual_3_dire_matches,emp_local_igual_3_dire_matches,los_local_igual_3_dire_matches,vict_local_dif_3_dire_matches,emp_local_dif_3_dire_matches,los_local_dif_3_dire_matches,cuota_local,cuota_empate,cuota_visitante
4365,2020,liga-dimayor-apertura,Semifinal,NaN,América de Cali,CD Junior,0,NaN,0,0,...,0.000000,0.333333,0.333333,0.333333,0.333333,0.666667,0.333333,NaN,NaN,NaN
2907,2017,liga-dimayor-apertura,todos contra todos,5.0,Tigres FC,Deportivo Cali,1,Jorge Duarte,15,2,...,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3467,2018,liga-dimayor-apertura,todos contra todos,18.0,Millonarios,Atletico Huila,6,Jhon Hinestroza,10,4,...,0.333333,0.666667,0.333333,0.666667,0.333333,0.666667,0.333333,NaN,NaN,NaN
3506,2018,liga-dimayor-clausura,todos contra todos,1.0,Jaguares De Córdoba,Once Caldas,0,Oscar Gómez,19,2,...,1.000000,0.333333,0.333333,0.333333,0.000000,0.333333,0.000000,NaN,NaN,NaN
1000,2012,liga-dimayor-apertura,todos contra todos,12.0,Once Caldas,Envigado FC,6,Juan Carlos Gamarra,16,9,...,0.666667,0.666667,0.333333,0.666667,0.500000,0.000000,0.500000,NaN,NaN,NaN


In [24]:
scores = scores.drop(["cuota_local", "cuota_empate", "cuota_visitante"], axis=1)

# 1. CatBoost Multiclass

In [25]:
scores_new = scores # [scores["fase"]=="todos contra todos"]
y_train = scores_new["result"]
x_train = scores_new.drop(["result"], axis=1)
x_train

,year,torneo,fase,jornada,local,visitante,dia,arbitro,pos_local,pos_visitante,...,los_local_local_3_matches,los_local_visit_3_matches,los_visit_local_3_matches,los_visit_visit_3_matches,vict_local_igual_3_dire_matches,emp_local_igual_3_dire_matches,los_local_igual_3_dire_matches,vict_local_dif_3_dire_matches,emp_local_dif_3_dire_matches,los_local_dif_3_dire_matches
0,2009,liga-dimayor-clausura,todos contra todos,1.0,Real Cartagena,Independiente Santa Fe,6,Carlos Anaya,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,liga-dimayor-clausura,todos contra todos,1.0,Boyaca Chico FC,Atletico Huila,6,Luis Sánchez,11,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,liga-dimayor-clausura,todos contra todos,1.0,Once Caldas,Deportivo Pereira,6,Weimar Hernández,16,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009,liga-dimayor-clausura,todos contra todos,1.0,Cúcuta Deportivo,América de Cali,6,Hárold Perilla,14,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009,liga-dimayor-clausura,todos contra todos,1.0,Tolima,Quindío,6,Óscar Reyes,5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,2024,liga-dimayor-finalizacion,todos contra todos,9.0,Boyaca Chico FC,Independiente Santa Fe,1,Luis Delgado,17,6,...,0.333333,0.666667,0.333333,0.666667,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000
5769,2024,liga-dimayor-finalizacion,todos contra todos,9.0,Envigado FC,Patriotas Boyaca,1,Diego Ulloa,12,19,...,0.333333,0.333333,0.000000,1.000000,0.666667,0.333333,0.666667,0.333333,0.666667,0.333333
5770,2024,liga-dimayor-finalizacion,todos contra todos,9.0,Fortaleza FC,Alianza FC,2,Bismark Santiago,9,16,...,0.000000,0.666667,0.666667,0.333333,NaN,NaN,NaN,NaN,NaN,NaN
5771,2024,liga-dimayor-finalizacion,todos contra todos,9.0,Millonarios,Once Caldas,2,Wilmar Roldán,10,13,...,0.333333,0.666667,0.333333,0.333333,1.000000,0.000000,1.000000,0.333333,0.000000,0.333333


In [26]:
clf = CatBoostClassifier(
    iterations=450, 
    learning_rate=0.01,
    custom_loss=['AUC', 'Accuracy'], 
    loss_function='MultiClass'
)

accuracies = []
cat_features = ['torneo', 'fase', 'local', 'visitante']
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)):
    X_train_fold, y_train_fold = x_train.iloc[train_idx], y_train.iloc[train_idx]
    X_val_fold, y_val_fold = x_train.iloc[val_idx], y_train.iloc[val_idx]

    clf.fit(X_train_fold, y_train_fold, 
            cat_features=cat_features, 
            eval_set=(X_val_fold, y_val_fold), 
            verbose=False,
            plot=True
    )
    pred = clf.predict(X_val_fold)
    accuracy = accuracy_score([str(x) for x in y_val_fold], pred)
    accuracies.append(accuracy)
    print(f"Fold{fold} acuracy ={accuracy}")

print(f"Average acuracy ={np.mean(accuracies)}")




CatBoostError: Invalid type for cat_feature[non-default value idx=122,feature_idx=7]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:

clf.fit(x_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val_fold, y_val_fold), 
        verbose=False,
)
soft_pred = clf.predict_proba(x_train)
hard_pred = clf.predict(x_train)
predictions = pd.concat([x_train, y_train], axis=1)

report = classification_report(y_train, hard_pred)
print("Classification Report:\n", report)




Classification Report:
               precision    recall  f1-score   support

      Empate       0.66      0.04      0.08      1720
       Local       0.49      0.97      0.65      2721
   Visitante       0.51      0.12      0.19      1332

    accuracy                           0.50      5773
   macro avg       0.56      0.38      0.31      5773
weighted avg       0.55      0.50      0.38      5773



In [ ]:
train_dataset = Pool(data=x_train,
                     label=y_train,
                     cat_features = cat_features)

clf.fit(train_dataset, verbose=False)


conf_matrix = pd.DataFrame(get_confusion_matrix(clf, train_dataset), clf.classes_, clf.classes_)
conf_matrix
# Rows: Actual values
# Columns: Predicted values

,Empate,Local,Visitante
Empate,73.0,1565.0,82.0
Local,11.0,2641.0,69.0
Visitante,26.0,1148.0,158.0


In [ ]:
joblib.dump(clf, 'catboost_multiclass_model.pkl')


['catboost_multiclass_model.pkl']